In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import os
import argparse
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import joblib
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
# TOO create TabularDataset using TabularDatasetFactory
# Data is located at:
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(url)

In [ ]:
run = Run.get_context()

In [ ]:
def clean_data(data):
    # Dict for cleaning data
    months = {"jan":1, "feb":2, "mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12}
    weekdays = {"mon":1,"tue":2,"wed":3,"thu":4,"fri":5,"sat":6,"sun":7}
    
    # clean and OneHotEncode data
    x_df = data.to_pandas_dataframe().dropna()
    jobs = pd.get_dummies(x_df.job, prefix="job")
    x_df.drop("job", inplace=True, axis=1)
    x_df = x_df.join(jobs)
    x_df["marital"] = x_df.marital.apply(lambda s: 1 if s =='married' else 0)
    x_df["default"] = x_df.default.apply(lambda s: 1 if s =='yes' else 0)
    x_df['housing'] = x_df.housing.apply(lambda s: 1 if s =='yes' else 0)
    x_df["loan"] = x_df.loan.apply(lambda s: 1 if s =='yes' else 0)
    contact = pd.get_dummies(x_df.contact, prefix='contact')
    x_df.drop("contact", inplace=True, axis=1)
    x_df = x_df.join(contact)
    education = pd.get_dummies(x_df.education, prefix='education')
    x_df.drop("education", inplace=True, axis=1)
    x_df = x_df.join(education)
    x_df["month"] = x_df.month.map(months)
    x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
    x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s =='success' else 0)
    
    y_df = x_df.pop("y").apply(lambda s: 1 if s =='yes' else 0)
    
    return x_df, y_df

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state =42)